In [1]:
#import PyPlot
#import Seaborn
import JLD2
import Distributions

In [2]:
include("src\\NGSIM.jl")

find_pairs

In [3]:
# td = load_trajdata(1);
JLD2.@load "td.jld" td
rd = NGSIM.ROADWAY_101

(S, id_lookup) = td_sparse(td)
;

In [36]:
fs = [
    s::VehicleState -> s.v * cos(s.posF.ϕ), 
    s::VehicleState -> s.v * sin(s.posF.ϕ),
    s::VehicleState -> length(rd[s.posF.roadind.tag.segment].lanes) - s.posF.roadind.tag.lane + 1
]

@time (hmm, hmm_data) = init_em_problem(S, fs);

 12.341416 seconds (28.34 M allocations: 849.452 MiB, 1.78% gc time)


In [ ]:
function update_params!(hmm::HMM, hmm_data::HMM_Data)
    T = hmm_data.T
    M = hmm.M
    K = hmm.K
    E = hmm_data.E

    π0 = hmm.π0
    A = hmm.A
    bΔ = hmm.bΔ
    c = hmm.c
    μs = hmm.μs
    Σs = hmm.Σs

    S = hmm_data.S
    YΓ = hmm_data.YΓ
    γ = hmm_data.γ
    γ_mix = hmm_data.γ_mix

    γ_mix_sum = squeeze(sum(γ_mix, 3), 3)
    γ_sum = squeeze(sum(γ, 2), 2)
    
    #
    # π0
    #
    fill!(π0, 0)
    for e = 1:E # per trajectory
        # where the e-th trajectory starts in the data
        start_idx = S.colptr[e]

        (start_idx ≥ T) && break

        π0 += γ[:, start_idx]
    end

    normalize!(π0, 1)
    map!(log, hmm_data.log_π0, π0)

    
    #
    # a
    #
    fill(A, 0)
    for e = 1:E # per trajectory
        # where the e-th trajectory starts in the data
        start_idx = S.colptr[e]
        end_idx = S.colptr[e+1] - 2
    
        (start_idx > T) && break

        for t in start_idx:end_idx
            (t > T) && break
            A += ξ[:, :, t]
        end
    end
    
    
    
    #
    # b
    #
    
    
    #
    # c
    #
    c[:] = γ_mix_sum ./ γ_sum'
    c ./= sum(c, 1)
    map!(log, hmm_data.log_c, c)

    #
    # μ
    #
    fill!(μs, 0)
    @inbounds for t = 1:T
        o = YΓ[:, t]

        for k in 1:K
            for m in 1:M
                μs[:, m, k] += γ_mix[m, k, t] * o
            end
        end
    end

    @inbounds for k in 1:K
        for m in 1:M
            μs[:, m, k] ./= γ_mix_sum[m, k]
        end
    end

    
    #
    # Σ
    #
    fill!(Σs, 0)
    @inbounds for t = 1:T
        o = YΓ[:, t]

        for k in 1:K
            for m in 1:M
                Σs[:, :, m, k] += γ_mix[m, k, t] * o * o'
            end
        end
    end

    @inbounds for k in 1:K
        for m in 1:M
            Σs[:, :, m, k] ./= γ_mix_sum[m, k]
        end
    end

    @inbounds for m in 1:M
        for k in 1:K
            hmm_data.invΣs[:, :, m, k] = inv(Σs[:, :, m, k])
            hmm_data.logdetΣs[m, k] = logdet(Σs[:, :, m, k])
        end
    end
end


In [75]:
function EM_iters(hmm, hmm_data)
    
end

data_likelihood! (generic function with 1 method)